# Mission to Mars

In [34]:
from splinter import Browser
from bs4 import BeautifulSoup
#import requests
import time
import pandas as pd
import pymongo
from flask import Flask, render_template, redirect

In [4]:
!which chromedriver

/usr/local/bin/chromedriver


In [5]:
# Set up Splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## Nasa Mars News

In [6]:
# Visit URL of page to be scraped through splinter
url = 'https://mars.nasa.gov/news/'

browser.visit(url)

In [10]:
# Create html object
html = browser.html

# Parse with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

In [21]:
# Scrape and collect the latest News Title and Paragraph Text. 
# Assign the text to variables that you can reference later.

# Alternatively: (Title is coming up wrong)
# news_title = soup.find_all('div', class_ = 'content_title')find('a').text
# news_p = soup.find('div', class_ = 'article_teaser_body').text

news_title = soup.find_all('div', class_ = 'content_title')[1].text
news_p = soup.find_all('div', class_ = 'article_teaser_body')[0].text

print("Title: ", news_title)
print("Paragraph: ", news_p)

Title:  Independent Review Indicates NASA Prepared for Mars Sample Return Campaign
Paragraph:  NASA released an independent review report Tuesday indicating the agency is well positioned for its Mars Sample Return campaign to bring pristine samples from Mars to Earth for scientific study.


## JPL Mars Space Images - Featured Image

In [35]:
# Visit URL of page to be scraped through splinter
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(jpl_url)

In [36]:
#Click on the the button go to the full image
full_image = browser.find_by_id("full_image")
full_image.click()
browser.is_element_present_by_text("more info")
time.sleep(1)
full_size = browser.links.find_by_partial_text("more info")
full_size.click()

In [24]:
# Create html object
jpl_html = browser.html

# Parse with BeautifulSoup
jpl_soup = BeautifulSoup(jpl_html, 'html.parser')

In [41]:
#Retrieve backgroud image url from style tag/scrape url
img_url = jpl_soup.find("img", class_ = "main_image")["src"]
featured_img_url = "https://www.jpl.nasa.gov" + img_url
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14762_hires.jpg'

## Mars Facts

In [42]:
#Visit the Mars Facts webpage
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)

In [43]:
# Use Pandas to parse the URL
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [46]:
#Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
facts_df = tables[0]

#Assign column names
facts_df.columns = ["Description", "Value"]

#Formatting
facts_df["Description"] = facts_df["Description"].str.replace(":", "")

facts_df

,Description,Value
0,Equatorial Diameter,"6,792 km"
1,Polar Diameter,"6,752 km"
2,Mass,6.39 × 10^23 kg (0.11 Earths)
3,Moons,2 (Phobos & Deimos)
4,Orbit Distance,"227,943,824 km (1.38 AU)"
5,Orbit Period,687 days (1.9 years)
6,Surface Temperature,-87 to -5 °C
7,First Record,2nd millennium BC
8,Recorded By,Egyptian astronomers


In [45]:
#Use Pandas to convert the data to a HTML table string
facts_html = facts_df.to_html()

print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Re

## Mars Hemispheres

In [51]:
# Visit url to be scraped through splinter 
hems_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hems_url)

In [52]:
# Create HTML object & Parse with BeautifulSoup
hems_html = browser.html

hems_soup = BeautifulSoup(hems_html, "html.parser")

In [53]:
# Create empty list for saving dictionaries title, url
hemisphere_image_urls = []

products = hems_soup.find('div', class_='result-list')

hemispheres = products.find_all('div',{'class':'item'})

In [50]:
# Loop through to append title and url
for hems in hemispheres:
    title = hems.find("h3").text
    title = title.replace(" Enhanced", "")
    img_link = hems.find("a")["href"]
    image_url = "https://astrogeology.usgs.gov/" + img_link 
    
    browser.visit(image_url)
    hems_html = browser.html
    hems_soup = BeautifulSoup(hems_html, "html.parser")
    downloads = hems_soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [19]:
for i in range(len(hemisphere_image_urls)):
    print(hemisphere_image_urls[i]['title'])
    print(hemisphere_image_urls[i]['img_url'] + '\n')

Cerberus Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg

Schiaparelli Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg

Syrtis Major Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg

Valles Marineris Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg

Cerberus Hemisphere
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg

Schiaparelli Hemisphere
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg

Syrtis Major Hemisphere
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg

Valles Marineris Hemisphere
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg

